## SAY SOMETHING HERE

A company-year long format xt panel data table, 2005-2016 
 n = 287,829 observations (46,412 firms)
	(third as many companies in 2016 as in earlier years)
 - ID variables	comp_id	(numerical) company identifier
- year	(calendar) year

 other important variables
 
- exit_year	year of exit (missing if still in business on 2016.12.31) 
- sales		sales in year (EUR)
- curr_assets	current assets at end of year (EUR)
- ind2		2-digit NACE industry code

DEFAULT =  Existed in 2014 (sales > 0), but did not exists in 2015 (sales is 0 or missing)

In [245]:
import pandas as pd
import numpy as np

In [246]:
data = pd.read_csv('cs_bisnode_panel.zip')
data

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1.001034e+06,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1.001034e+06,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1.001034e+06,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1.001034e+06,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1.001034e+06,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287824,4.641209e+11,2011-01-01,2011-12-31,NaN,0.000000,1807.407349,3.703704,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287825,4.641209e+11,2012-01-01,2012-12-31,NaN,0.000000,1518.518555,96.296295,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287826,4.641209e+11,2013-01-01,2013-12-31,NaN,0.000000,988.888916,77.777779,0.0,0.0,0.0,...,NaN,NaN,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287827,4.641209e+11,2014-01-01,2014-12-31,NaN,0.000000,644.444458,233.333328,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN


In [247]:
data = data.loc[data['ind2'] == 26]
data

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
960,6.538183e+06,2005-01-01,2005-12-31,NaN,792.592590,6237.037109,348.148163,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
961,6.538183e+06,2006-01-01,2006-12-31,NaN,196.296295,6007.407227,451.851837,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
962,6.538183e+06,2007-01-01,2007-12-31,NaN,233.333328,9007.407227,2637.037109,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
963,6.538183e+06,2008-01-01,2008-12-31,NaN,422.222229,7707.407227,1651.851807,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
964,6.538183e+06,2009-01-01,2009-12-31,NaN,1525.925903,18196.296875,8714.814453,18.518518,0.0,-18.518518,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287439,4.628231e+11,2012-01-01,2012-12-31,NaN,359.259247,21662.962891,19481.482422,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.333333
287440,4.628231e+11,2013-01-01,2013-12-31,NaN,3159.259277,15488.888672,21440.740234,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.298611
287441,4.628231e+11,2014-01-01,2014-12-31,NaN,2581.481445,70059.257812,74911.109375,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.486111
287442,4.628231e+11,2015-01-01,2015-12-31,NaN,2177.777832,46062.964844,49844.445312,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.562500


In [249]:
#criteria for the holdout sample so I can mask the training sample with !holdout_criteria
holdout_criteria = (data['year'] == 2014) & ((data['sales'] >=1000) & (data['sales']<=10000000))

In [250]:
#holdout sample creation
holdout = data[holdout_criteria]
print(f'Holdout shape: {holdout.shape}')
holdout.head()

Holdout shape: (1037, 48)


,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
969,6538183.0,2014-01-01,2014-12-31,NaN,1018.518494,17022.222656,3040.740723,0.0,0.0,0.0,...,male,Domestic,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
1128,8416055.0,2014-01-01,2014-12-31,NaN,425.925934,105740.742188,6918.518555,0.0,0.0,0.0,...,female,Domestic,2651.0,26.0,2.0,1,Central,1995-08-28,NaN,0.083333
1467,12428378.0,2014-01-01,2014-12-31,NaN,0.000000,0.000000,9277.777344,0.0,0.0,0.0,...,male,Domestic,2660.0,26.0,2.0,2,Central,1991-06-27,NaN,0.083333
1706,17776540.0,2014-01-01,2014-12-31,NaN,48.148148,142296.296875,164237.031250,0.0,0.0,0.0,...,mix,Domestic,2620.0,26.0,2.0,1,Central,1997-07-01,NaN,0.305556
1735,18626760.0,2014-01-01,2014-12-31,NaN,5755.555664,95185.187500,45766.667969,0.0,0.0,0.0,...,male,Domestic,2670.0,26.0,2.0,2,West,2010-11-15,NaN,0.159091


In [251]:
training = data[~holdout_criteria]
training

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
960,6.538183e+06,2005-01-01,2005-12-31,NaN,792.592590,6237.037109,348.148163,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
961,6.538183e+06,2006-01-01,2006-12-31,NaN,196.296295,6007.407227,451.851837,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
962,6.538183e+06,2007-01-01,2007-12-31,NaN,233.333328,9007.407227,2637.037109,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
963,6.538183e+06,2008-01-01,2008-12-31,NaN,422.222229,7707.407227,1651.851807,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
964,6.538183e+06,2009-01-01,2009-12-31,NaN,1525.925903,18196.296875,8714.814453,18.518518,0.0,-18.518518,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287438,4.628231e+11,2011-01-01,2011-12-31,NaN,1462.963013,16007.407227,6392.592773,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.234848
287439,4.628231e+11,2012-01-01,2012-12-31,NaN,359.259247,21662.962891,19481.482422,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.333333
287440,4.628231e+11,2013-01-01,2013-12-31,NaN,3159.259277,15488.888672,21440.740234,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.298611
287442,4.628231e+11,2015-01-01,2015-12-31,NaN,2177.777832,46062.964844,49844.445312,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.562500
